# Détection, Segmentation et Analyse d'Anévrismes Cérébraux

## Installation

Plusieurs paquets sont nécessaires pour exécuter le script en lui-même ainsi que le notebook.

`pip install pydicom ipympl opencv-python scikit-image scipy`

Afin d'exécuter le script Python avec l'interface complète, il faut exécuter la commande suivante :

`python main.py`

Le dossier ImgTP doit se trouver à la racine du dossier où se trouve main.py

In [2]:
%matplotlib widget
import copy
from helpers import *
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive
from IPython.display import display, clear_output, HTML, Video

# Lecture du DICOMDIR et traitement des images

Dans un premier temps, nous avons recherché un outil en Python capable de lire des fichiers DICOM et d'en extraire les informations nécessaires. Le paquet `pydicom` permet ceci.

<!-- Description de l'installation -->

Une fois ce paquet installé, nous pouvons lire notre jeu de données. Celui-ci contient des informations à plusieurs niveaux : pour chaque patient, il peut y avoir une ou plusieurs expérience(s) contenant chacune une ou plusieurs série(s) d'images. Dans notre cas, nous n'avons qu'une seule série. Nous récupérons les chemins de ces images puis les lisons avec la fonction `dcmread` du paquet `pydicom`.

<!-- On a remarqué que sur les 500 et quelques images qu'on a, il y a une répétition toutes les 48 images -->

Ces images sont en niveaux de gris sur 16 bits.

In [3]:
dataset_path = "ImgTP/Ge1CaroG/MR_3DPCA"
ds = load_dataset(dataset_path)

# 0018,1090  Cardiac Number of Images: 12
nb_image_sets = 12

# Select the first image set (must be < 12)
starting_image_set = 0

In [4]:
data = {}

# Iterate through the PATIENT records
for patient in ds.patient_records:
    # Find all the STUDY records for the patient
    studies = [ii for ii in patient.children if ii.DirectoryRecordType == "STUDY"]

    for study in studies:
        # Find all the SERIES records in the study
        all_series = [ii for ii in study.children if ii.DirectoryRecordType == "SERIES"]

        for series in all_series:
            # Find all the IMAGE records in the series
            images = [ii for ii in series.children if ii.DirectoryRecordType == "IMAGE"]

            # Get the absolute file path to each instance
            # Each IMAGE contains a relative file path to the root directory
            elems = [ii["ReferencedFileID"] for ii in images]
            # Make sure the relative file path is always a list of str
            paths = [[ee.value] if ee.VM == 1 else ee.value for ee in elems]
            paths = [Path(*p) for p in paths]

            images = []

            i = 0

            total_images = len(paths)
            max_images = int(total_images / nb_image_sets)

            # List the instance file paths
            for idx in range(starting_image_set * max_images, (starting_image_set + 1) * max_images):
                p = paths[idx]
                img = dcmread(Path(dataset_path).joinpath(p))
                images.append((img, p))

            data['images'] = images

## Suppression du bruit

### Explications

La suppression du bruit peut s'effectuer par l'utilisation de différents filtres. Parmi ceux-ci on retrouve par exemple le filtre bilatéral, le débruitage par patchs (Non-Local Means) ou encore le filtre médian.

### Déclaration des fonctions

Nous utilisons ici les fonctions mises à disposition par skimage et scipy.

In [5]:
from skimage.restoration import denoise_nl_means, estimate_sigma, denoise_bilateral

def apply_simple_denoise(img, denoise_filter=ndimage.median_filter, kernel_size=3):
    new_img = denoise_filter(img.pixel_array, kernel_size)

    return new_img


def apply_non_local_means(img, kernel=5, window_search=13):
    # Retain original data type
    orig_dtype = img.pixel_array.dtype

    # Convert from [0; max] to [0; 1] as it is required by denoise_nl_means
    upper_bound = np.max(img.pixel_array)
    img_as_float = img.pixel_array / upper_bound

    sigma_est = np.mean(estimate_sigma(img_as_float, multichannel=False))

    new_img = denoise_nl_means(img_as_float, h=sigma_est, fast_mode=True, patch_size=kernel,
                               patch_distance=window_search)

    # Convert back to [0; max]
    new_img *= upper_bound

    return new_img.astype(orig_dtype)


def apply_bilateral_filtering(img, d=15, sigmacolor=75, sigmacoordinate=75):
    # Retain original data type
    orig_dtype = img.pixel_array.dtype

    # Convert from [0; max] to [0; 1] as it is required by denoise_bilateral
    upper_bound = np.max(img.pixel_array)
    img_as_float = img.pixel_array / upper_bound

    new_img = denoise_bilateral(img_as_float, win_size=d, sigma_color=sigmacolor, sigma_spatial=sigmacoordinate)

    # Convert back to [0; max]
    new_img *= upper_bound

    return new_img.astype(orig_dtype)

Pour chaque image, on effectue la suppression du bruit avec chacun des filtres précédemment cités puis on stocke toutes ces nouvelles images afin de les afficher et les comparer.

In [6]:
images = data['images']
denoised_images = {}

median_images = []
bilateral_images = []
non_local_images = []

for image, p in images:
    median = apply_simple_denoise(image, kernel_size=3)
    bilateral = apply_bilateral_filtering(image, 4, 35, 35)
    non_local = apply_non_local_means(image)

    median_images.append(median)
    bilateral_images.append(bilateral)
    non_local_images.append(non_local)

denoised_images['median'] = median_images
denoised_images['bilateral'] = bilateral_images
denoised_images['non_local'] = non_local_images

data['denoised_images'] = denoised_images

### Affichage des résultats de suppression de bruit

In [17]:
image_sets = [
    {
        'images': [image.pixel_array for image, _ in data['images']],
        'title': 'Original',
        'shown': True
    },
    {
        'images': data['denoised_images']['median'],
        'title': 'Median Filter',
        'shown': True
    },
    {
        'images': data['denoised_images']['bilateral'],
        'title': 'Bilateral Filter',
        'shown': True
    },
    {
        'images': data['denoised_images']['non_local'],
        'title': 'Non Local Means',
        'shown': True
    }
]

output = widgets.Output()

fig = None

with output:
    plt.close(fig)
    fig = plt.figure()

# Hide figure header
fig.canvas.header_visible = False
ls = []
current_image_slider = 0
shown_image_sets = [image_set for image_set in image_sets if image_set['shown']]
zoom = 2
output_h, output_w = 0, 0

# Executed on image slider change
def update_images(change):
    global shown_image_sets, ls
    current_image_slider = change.new

    for idx, image in enumerate(ls):
        image.set_data(shown_image_sets[idx]['images'][change.new])


# Executed on zoom slider change
def update_zoom(change):
    global zoom, fig
    zoom = change.new
    fig.set_size_inches(output_w * zoom / 100, output_h * zoom / 100, forward=True)


def plot_images(fig, image_sets):
    global shown_image_sets, output_h, output_w
    shown_image_sets = [image_set for image_set in image_sets if image_set['shown']]
    nb_shown_image_sets = len(shown_image_sets)

    ncols = 4
    nrows = int(np.ceil(nb_shown_image_sets / float(ncols)))
    height, width = shown_image_sets[0]['images'][0].shape
    output_h, output_w = (height * nrows), (width * ncols)

    # Clear previous figure
    fig.clf()
    # Set figure size based on the number of image sets
    fig.set_size_inches(output_w * zoom / 100, output_h * zoom / 100, forward=True)
    fig.set_dpi(100)

    for idx in range(nb_shown_image_sets):
        ax = fig.add_subplot(nrows, ncols, idx + 1)
        image = ax.imshow(shown_image_sets[idx]['images'][current_image_slider], cmap=plt.cm.gray)
        ls.append(image)

        # Set title
        ax.title.set_text(shown_image_sets[idx]['title'])

        # Hide grid and x, y ticks
        ax.grid(False)
        plt.xticks([])
        plt.yticks([])

image_slider = widgets.IntSlider(
    value=0,
    min=0, max=47, step=1,
    continuous_update=True,
    description='Image',
    layout=widgets.Layout(width='99%')
)

zoom_slider = widgets.FloatSlider(
    value=zoom,
    min=1, max=3, step=0.1,
    continuous_update=False,
    description='Zoom',
    layout=widgets.Layout(width='99%')
)

image_slider.observe(update_images, 'value')
zoom_slider.observe(update_zoom, 'value')

# plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.tight_layout()

plot_images(fig, shown_image_sets)

widgets.VBox([output, image_slider, zoom_slider])

<!-- !["Denoising Example"](examples/denoising_example.png) -->

### Exemple de suppression du bruit sur l'image 16

<img src="examples/denoising_example.png" align="left"/>

Le filtre bilatéral semble flouter les contours tandis que le filtre Non-Local Means réduit moins bien le bruit que le filtre médian. On utilisera par la suite uniquement les résultats obtenus par le filtre médian.

Ce filtre réduit relativement bien le bruit tout en conservant les contours. Il est ici utilisé avec un kernel de taille 3x3.

<!-- Explication de pourquoi on utilise le filtre médian (meilleurs résultats, maintien des contours, ...) -->

## Segmentation du réseau vasculaire cérébral

Une fois le bruit supprimé de nos images, nous pouvons passer à la segmentation. Notre but est d'extraire les deux carotides et le tronc basilaire de nos images.

Cette fois encore, plusieurs possibilités : la segmentation par seuillage (Threshold), par marche aléatoire (Random Walker) ou par remplissage par diffusion (Flood Fill).

### Déclaration des fonctions

In [9]:
from skimage.segmentation import random_walker

def apply_random_walker(image):
    upper_bound = np.max(image)
    img_as_float = image / upper_bound
    img_as_float *= 2
    img_as_float -= 1

    markers = np.zeros(img_as_float.shape, dtype=np.uint)
    markers[img_as_float < -0.90] = 1
    markers[img_as_float > 0.90] = 2

    return random_walker(img_as_float, markers, beta=10, mode='bf')


def apply_flood_fill(image, starting_coordinates, tolerance):
    upper_bound = np.max(image)
    img_as_float = image / upper_bound
    mask = flood(img_as_float, starting_coordinates, tolerance=tolerance)
    mask = mask.astype('uint16')

    return mask


def apply_threshold(image):
    # thresh = threshold_mean(image)
    return image > 300

In [10]:
denoised_images = data['denoised_images']['median']
segmented_images = {}

threshold_images = []
random_walker_images = []
fills = {}

for median_image in denoised_images:
    thresh = apply_threshold(median_image)
    random_walker_img = apply_random_walker(median_image)

    for tol in np.linspace(0.0, 0.2, 12):
        str_tol = "{:.2f}".format(tol)

        if str_tol not in fills:
            fills[str_tol] = []

        fill = apply_flood_fill(median_image, (71, 76), tol)
        fills[str_tol].append(fill)

    threshold_images.append(thresh)
    random_walker_images.append(random_walker_img)

segmented_images['threshold'] = threshold_images
segmented_images['random_walker'] = random_walker_images
segmented_images['flood_fill'] = fills

data['segmented_images'] = segmented_images

### Affichage des résultats de segmentation

In [20]:
image_sets = [
    {
        'images': [image.pixel_array for image, _ in data['images']],
        'title': 'Original',
        'shown': True
    },
    {
        'images': data['denoised_images']['median'],
        'title': 'Median Filter',
        'shown': True
    },
    {
        'images': data['segmented_images']['threshold'],
        'title': 'Threshold',
        'shown': False
    },
    {
        'images': data['segmented_images']['random_walker'],
        'title': 'Random Walker',
        'shown': False
    }
]

for tol, fill in data['segmented_images']['flood_fill'].items():
    image_sets.append({
        'images': fill,
        'title': 'Flood Fill Tol {}'.format(tol),
        'shown': False
    })

nb_image_sets = len(image_sets)

output = widgets.Output()

fig = None

with output:
    plt.close(fig)
    fig = plt.figure()

# Hide figure header
fig.canvas.header_visible = False
ls = []
current_image_slider = 0
shown_image_sets = [image_set for image_set in image_sets if image_set['shown']]
zoom = 2
output_h, output_w = 0, 0

# Executed on image slider change
def update_images(change):
    global shown_image_sets, ls, current_image_slider
    current_image_slider = change.new

    for idx, image in enumerate(ls):
        image.set_data(shown_image_sets[idx]['images'][change.new])

# Executed on zoom slider change
def update_zoom(change):
    global zoom, fig
    zoom = change.new
    fig.set_size_inches(output_w * zoom / 100, output_h * zoom / 100, forward=True)

# Executed on image sets selection
def update_image_sets(change):
    global image_sets, ls, fig
    ls.clear()

    for image_set in image_sets: image_set['shown'] = False
    
    for idx in change.owner.index:
        image_sets[idx]['shown'] = True
    
    plot_images(fig, image_sets)

def plot_images(fig, image_sets):
    global shown_image_sets, output_w, output_h, current_image_slider
    shown_image_sets = [image_set for image_set in image_sets if image_set['shown']]
    nb_shown_image_sets = len(shown_image_sets)
    
    ncols = 6 if nb_shown_image_sets > 6 else nb_shown_image_sets
    nrows = int(np.ceil(nb_shown_image_sets / float(ncols)))
    height, width = shown_image_sets[0]['images'][0].shape
    output_h, output_w = (height * nrows), (width * ncols)
    
    # Clear previous figure
    fig.clf()
    # Set figure size based on the number of images
    fig.set_size_inches(output_w * zoom / 100, output_h * zoom / 100, forward=True)
    fig.set_dpi(100)
    
    for idx in range(nb_shown_image_sets):
        ax = fig.add_subplot(nrows, ncols, idx + 1)
        image = ax.imshow(shown_image_sets[idx]['images'][current_image_slider], cmap=plt.cm.gray)
        ls.append(image)
        
        # Set title
        ax.title.set_text(shown_image_sets[idx]['title'])
        
        # Hide grid and x, y ticks
        ax.grid(False)
        plt.xticks([])
        plt.yticks([])
    
image_slider = widgets.IntSlider(
    value=0, 
    min=0, max=47, step=1,
    continuous_update=True,
    description='Image',
    layout=widgets.Layout(width='99%')
)

zoom_slider = widgets.FloatSlider(
    value=zoom,
    min=1, max=3, step=0.1,
    continuous_update=False,
    description='Zoom',
    layout=widgets.Layout(width='99%')
)

image_set_selection = widgets.SelectMultiple(
    # Add all image sets as options 
    options=[image_set['title'] for image_set in image_sets],
    # Select by default the ones that are set as 'shown'
    value=[image_set['title'] for image_set in shown_image_sets],
    disabled=False,
    layout=widgets.Layout(width='99%', align_items='stretch')
)

image_slider.observe(update_images, 'value')
zoom_slider.observe(update_zoom, 'value')
image_set_selection.observe(update_image_sets, 'value')

# plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.tight_layout()

plot_images(fig, shown_image_sets)

widgets.VBox([image_set_selection, output, image_slider, zoom_slider])

### Exemple de segmentation sur l'image 16

<img src="examples/segmentation_example.png" align="left"/>

En dépit des segmentations utilisées, nous avons retenu la méthode du Flood Fill puisque le Threshold et le Random Walker ne nous permettaient pas d'avoir une segmentation propre et correcte.

En effet, le Threshold ne se fiait qu'à l'intensité du pixel ce qui prenait également les pixels trop proches en valeurs de gris (par exemple, le contour du crâne) et était donc trop sensible.  
Quant au Random Walker, il affichait une sphère et n'isolait pas du tout l'anévrisme ce qui n'était pas du tout ce que nous voulions.

Ainsi, le flood fill fut la seule méthode qui nous donna quelque chose de convenable à la vue des résultats que nous pouvons observer.  
Cette méthode a cependant le désavantage de nécessiter la sélection d'une « graine » (seed) pour commencer le flood fill. Celle-ci peut être sélectionnée par clic de souris.

# Segmentation par Flood Fill

Dans un premier temps, nous avons créé une interface permettant de cliquer sur une zone de l'image afin de définir la graine du flood fill.  
L'algorithme de flood fill n'est disponible qu'en 2D. Nous travaillons ici avec 48 images 2D et cette segmentation doit être effectuée pour chacune des coupes, il faut donc convertir cet algorithme pour qu'il fonctionne sur plusieurs images tout en gardant une connexité de la segmentation.

## Première solution


### Principe 

Le flood fill est effectué sur une image, nous obtenons un masque que l'on applique à l'image précédente/suivante.
À partir des valeurs de gris contenues dans ce masque, nous calculons la moyenne des pixels et déterminons la nouvelle graine parmi les pixels proches de cette moyenne avec une certaine tolérance.
Ainsi, le masque calculé pour l'image 16 sera appliqué à l'image 17, puis une nouvelle graine est déterminée à partir de ces valeurs. L'utilisation du masque déjà calculé sur la prochaine image permet d'assurer la connexité de la segmentation - en effet, le choix de la graine ne peut se faire que parmi les points communs aux deux images. Ce procédé fonctionne également dans le sens inverse : le masque de l'image 16 est appliqué à l'image 15, puis celui de l'image 15 à l'image 14 et ainsi de suite.

### Problèmes

L'approche par moyenne de valeur de gris pose un problème : cette moyenne change graduellement à mesure que l'on progresse dans les images. Ceci est dû à la fois à la tolérance du flood fill qui permet d'étendre le masque au-delà de la valeur de la graine initiale, mais également et surtout à la méthode de choix de la nouvelle graine. Le masque `n - 1` étant appliqué à l'image `n`, il se peut que l'on retrouve soudainement des valeurs de gris beaucoup plus sombres que celles de départ, et la moyenne s'en verrait alors fortement impactée.
C'est ce que nous observons sur la vidéo ci-dessous lorsque nous approchons du début de la série d'images.

### Résultats

<video src="examples/first_flood_fill.webm" type="video/webm" controls loop></video>

## Deuxième solution

### Principe

Le principe de sélection est le même : nous obtenons un masque appliqué à l'image précédente/suivante.  
Cette fois-ci, plutôt que de sélectionner la nouvelle graine en calculant la moyenne des niveaux de gris, on la sélectionne par le biais d'une tolérance par rapport à la valeur initiale de la précédente graine. Ainsi, si lors du premier flood fill, la graine avait une valeur de 300 alors la sélection de la graine de la prochaine image se fera par rapport à cette valeur, ± une tolérance configurable (**seed tolerance**). Cette méthode a pour avantage de ne pas dériver vers des valeurs plus sombres car nous n'introduisons plus de moyenne.  
De plus, afin que la sélection de la graine se fasse parmi les points déterminés les plus proches de la graine initiale, nous effectuons également une mesure de distance entre la graine initiale et toutes les graines candidates possibles (distance Euclidienne).  
Nous obtenons alors deux critères de sélection de la nouvelle graine : la valeur de gris et la distance à la graine initiale. Nous effectuons un tri basé sur ces deux critères, en privilégiant toutefois la sélection par valeur de gris ce qui nous donne alors un certain nombre de nouvelles graines triées en fonction de leur couleur puis de leur distance. Parmi celles-ci, nous prenons alors la première graine.


Nous appelons cet algorithme « Flood Fill évolutif ».

### Avantages

Le premier avantage de cette méthode est la garantie que l'on ne dérive pas vers des valeurs plus sombres ce qui nous permet de conserver une segmentation propre et ne contenant que les éléments qui nous intéressent.  
Le deuxième avantage est simplement la reproducibilité des résultats. En effet, là où la première solution effectuait une sélection aléatoire parmi les graines candidates, nous sélectionnons ici la graine en nous basant sur deux critères. Ainsi, une segmentation effectuée à partir de la même image et des mêmes coordonnées donnera toujours le même résultat, ce qui n'était pas garanti précédemment.

### Résultats

<video src="examples/second_flood_fill.webm" type="video/webm" controls loop></video>

Cette interface n'est complètement fonctionnelle qu'en dehors d'un Jupyter Notebook.  
Nous pouvons en voir un aperçu ci-dessous :

In [13]:
denoised_images = data['denoised_images']['median']
images = data['images']

subplots_slider(
    [['Original', [image.pixel_array for image, _ in images], {'type': 'original'}],
     ['Median Filter', denoised_images, {'type': 'median_filter'}],
     ['Mask', [np.zeros(denoised_images[0].shape)] * len(denoised_images), {'type': 'flood_fill', 'flood_fill_tolerance': globals.flood_fill_tolerance, 'seed_tolerance': globals.seed_tolerance}],
     ['Result', [np.zeros(denoised_images[0].shape)] * len(denoised_images), {'type': 'result'}],
     ['Skeleton', [np.zeros(denoised_images[0].shape)] * len(denoised_images), {'type': 'skeleton'}]],
    click_handler=select_region, zoom=3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Sélections multiples

L'algorithme de Flood Fill étant un algorithme à croissance de région connexe, une seule sélection ne nous permet pas d'obtenir la totalité de la segmentation.
Il a donc fallu modifier l'interface afin d'ajouter la possibilité de créer plusieurs segmentations qui sont ensuite fusionnées afin d'obtenir une segmentation finale.  
Chaque segmentation peut avoir des paramètres de tolérance différents en changeant simplement les valeurs avant d'effectuer une nouvelle segmentation.  
Il est également possible de supprimer une segmentation qui aurait par exemple mené à de mauvais résultats en cliquant sur le bouton `X`  
Nous obtenons finalement dans ce cas six segmentations qui sont toutes connexes. Il est cependant possible qu'elles ne soient pas connexes entre elles car on pourrait segmenter des parties non liées.

<img src="examples/six_segmentations.png" align="left" />

## Utilisation de l'outil

<video src="examples/demo_tool.webm" type="video/webm" controls loop></video>

## Visualisation de la segmentation en 3D

En utilisant l'outil de visualisation 3Dicom, nous avons pu charger toutes les images générées par notre outil de segmentation et visualiser l'anévrisme géant ainsi que le tronc basilaire et les carotides.

<video src="examples/3d_segmentation.webm" type="video/webm" controls loop></video>

# Création du squelette

Maintenant que nous avons obtenu une segmentation des parties nous intéressant, il nous faut désormais créer le squelette de cette segmentation.  
Obtenir celui-ci ne revient finalement qu'à effectuer une érosion maximale de morphologie sur les images, lorsque celles-ci n'ont pas de lien entre elles.  
Dans notre cas, nous souhaitons obtenir un squelette 3D, et simplement effectuer cette érosion sur chaque coupe en 2D donne de mauvais résultats.  
Le paquet `skimage` propose une fonction nommée « skeletonize_3d » qui prend en entrée un tableau 3D d'images 2D et effectue cette squelettonisation sur l'ensemble des images.

In [21]:
def resize_and_skeleton_3d(mask, factor, dilation_kernel=None):
    height, width = mask[0].shape
    dsize = (width * factor, height * factor)
    new_mask = [cv2.resize(image_mask, dsize, cv2.INTER_NEAREST) for image_mask in mask]

    skeletonized_mask = np.array([skeleton / 255 for skeleton in skeletonize_3d(np.array(new_mask))]).astype('uint8')

    if dilation_kernel is not None:
        skeletonized_mask = [skimage.morphology.binary_dilation(skeleton, dilation_kernel) for skeleton in skeletonized_mask]

    return np.array(skeletonized_mask).astype('uint16') * globals.max_gray_value

Nous utilisons la fonction `skeletonize_3d` sur le masque binaire car celle-ci n'accepte que des images en binaire.  
Dans un premier temps, nous obtenions un squelette très peu visible de par les faibles dimensions de nos images (150x192), nous avions alors décidé de redimensionner le masque binaire pour augmenter les dimensions du squelette.  
Hélas les résultats étaient très différents de ceux produits par la squelettonisation des images originales avec l'apparition de multiples branches qu'il aurait alors fallu ébarbuler.  
Nous avons alors décidé de garder le squelette des images de taille originale.

## Affichage des segmentations et du squelette

<img src="examples/six_segmentations_plus_skeleton.png" align="left" />

## Affichage du squelette 3D par Fiji

L'affichage en 2D des images du squelette n'a que peu de sens, ces images étant été générées par un algortihme 3D. Nous avons donc importé la liste des nouvelles images sur Fiji via « File -> Import -> Image Sequence » et selectionné l'une des images que nous avons enregistrées.  
Fiji s'occupait alors d'importer les images et il nous restait plus qu'à afficher les images en 3D par l'utilisation de « Image -> Stacks -> 3D project ». Hélas, le squelette s'en retrouve extrêmement pixelisé à cause de la faible dimension des images.  
Voici le résultat :

<img src="examples/skeleton_fiji.png" align="left" />

## Affichage du squelette 3D par 3Dicom

<video src="examples/3d_skeleton.webm" type="video/webm" controls loop></video>

# Analyse du squelette via FiJi

## Diamètre de l'anévrisme

Malgré les faibles dimensions du squelette, nous pouvons tout de même parvenir à analyser quelques détails. En effet, on peut même observer l'anévrisme. Pour davantage de clarté, nous allons colorier les pixels en blanc qui montrent donc l'anévrisme. Il se situe au sein d'un canal et nous pouvons ainsi l'identifier par une simple observation.  
Pour calculer la taille de l'anévrisme, nous allons calculer le nombre de pixels qui définissent sa taille ainsi que récupérer le « Pixel Spacing » et la « Slice Thickness » pour obtenir la taille en mm de chaque pixel.  
L'image ci-dessous correspond à la projection 2D de l'image 3D du squelette selon l'axe de rotation Y. Nous ne pouvons alors observer sur cette image que la largeur et la hauteur, et non la profondeur.

<img src="examples/anevrism_white_skeleton.png" align="left" />

In [37]:
ConstPixelSpacing = []
for patient in ds.patient_records:
    # Find all the STUDY records for the patient
    studies = [ii for ii in patient.children if ii.DirectoryRecordType == "STUDY"]

    for study in studies:
        # Find all the SERIES records in the study
        all_series = [ii for ii in study.children if ii.DirectoryRecordType == "SERIES"]

        for series in all_series:
            ConstPixelSpacing = (float(img.PixelSpacing[0]), float(img.PixelSpacing[1]), float(img.SliceThickness))

print("Dimensions d'un voxel\n\nLargeur (x) : {} mm\nHauteur (y) : {} mm\nProfondeur (z) : {} mm".format(round(ConstPixelSpacing[0], 3), round(ConstPixelSpacing[1], 3), round(ConstPixelSpacing[2], 3)))

Dimensions d'un voxel

Largeur (x) : 1.146 mm
Hauteur (y) : 1.146 mm
Profondeur (z) : 1.1 mm


Nous obtenons ainsi 1.1458 mm comme largeur et hauteur de pixel par le biais du code python ci-dessus permettant de récupérer les informations contenues dans l'en-tête des images DICOM.  
Il ne nous reste alors plus qu'à compter le nombre de pixels - celui-ci étant relativement faible - sur le squelette de l'anévrisme.  
On trouve alors les mesures maximales suivantes : 4 pixels en largeur et 5 pixels en hauteur.  

Nous obtenons alors :  
4 x pixel spacing = 4 x 1.1458 = 4.5832 mm en largeur  
5 x pixel spacing = 5 x 1.1458 = 5.7290 mm en hauteur  

Ces valeurs ne correspondent qu'aux dimensions du squelette et non aux dimensions réelles de l'anévrisme. Pour obtenir les valeurs réelles, il faudrait alors faire correspondre la segmentation obtenue avec le squelette par le biais d'une carte de distance.

## Diamètre en entrée des canaux

Sachant que nous n'avons qu'un seul pixel en entrée au niveau des canaux, il nous suffit alors de calculer la taille d'un pixel qui se trouve être :

1 x pixel spacing = 1 x 1.1458 = 1.1458 mm en hauteur/largeur

## Aire et périmètre de l'anévrisme

Grâce à Fiji, nous pouvons ensuite sélectionner la couleur blanche grâce à l'outil "Wand (tracing) tool" et sélectionner la zone pour permettre de sélectionner uniquement l'anévrisme. Nous allons ensuite utiliser « Analyze -> Set Measurement » et sélectionner les options suivantes :

<img src="examples/option_fiji_measurement.png" align="left" />

Maintenant que la sélection est faite et que les différentes options sont sélectionnées, nous cliquons sur « Analyze -> Measure » qui nous permet de lancer la mesure et obtenir alors les mesures de la zone que nous avons selectionnée. Ainsi, nous obtenons le tableau suivant qui nous permet d'obtenir l'aire et le périmètre du squelette de l'anévrisme que nous avons sélectionné.  
Nous obtenons ainsi une aire de 18.381 mm ainsi qu'un périmètre de 15.255 mm grâce aux résultats cités précédemment.

<img src="examples/measure_fiji.png" align="left" />